<a href="https://colab.research.google.com/github/praymesh/CAT-SEG-Test/blob/main/Dataset_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T
import numpy as np
import os

In [ ]:
dataset/
coco-stuff/
├── images/
│ ├── train2017/
│ │ ├── 000000000001.jpg
│ │ ├── ...
│ └── val2017/
├── annotations/
│ ├── train2017/
│ │ ├── 000000000001.png ← grayscale mask
│ └── val2017/
├── stuff_classes.txt ← class ID to name (e.g. "0 unlabeled")

In [33]:
!mkdir -p datasets/coco-stuff/images/train2017
!mkdir -p datasets/coco-stuff/images/val2017
!mkdir -p datasets/coco-stuff/annotations/train2017
!mkdir -p datasets/coco-stuff/annotations/val2017

In [34]:
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://calvin.inf.ed.ac.uk/wp-content/uploads/data/cocostuffdataset/stuffthingmaps_trainval2017.zip
# !wget http://calvin.inf.ed.ac.uk/wp-content/uploads/data/cocostuffdataset/stuff_classes.txt - link doest work|


--2025-06-20 20:51:18--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.16.183, 16.15.184.130, 52.217.130.1, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.16.183|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G   125MB/s    in 3m 27s  

2025-06-20 20:54:46 (89.0 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]

--2025-06-20 20:54:46--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.1.147, 52.216.8.59, 3.5.27.23, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.1.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  95.4MB/s    

In [35]:
#unzipping the dataset
#coc-stuff/images folder
!unzip -q train2017.zip -d datasets/coco-stuff/images/
!unzip -q val2017.zip -d datasets/coco-stuff/images/

#cocstuff/annotaions/masks
!unzip -q stuffthingmaps_trainval2017.zip -d datasets/coco-stuff/annotations/

#text classes in coco-stuff
#!unzip -q stuff_classes.txt -d datasets/coco-stuff/

For Class Labels

In [9]:
#dont use this :
!mkdir -p datasets/coco-stuff/
!wget https://github.com/nightrome/cocostuff/blob/master/labels.txt  # this downloads the webpage , ie the html file, code works but webpage gets downloaded as .txt file
!mv labels.txt datasets/coco-stuff/
!ls datasets/coco-stuff

open('datasets/coco-stuff/labels.txt').read() - #

--2025-06-20 20:33:38--  https://github.com/nightrome/cocostuff/blob/master/labels.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘labels.txt.6’

labels.txt.6            [ <=>                ] 174.48K  --.-KB/s    in 0.09s   

2025-06-20 20:33:38 (1.80 MB/s) - ‘labels.txt.6’ saved [178672]



!wget https://github.com/nightrome/cocostuff/blob/master/labels.txt --html webpage , thus downloading manually from same linked webpage :

In [21]:
# prompt: upload file from my pc to location

from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  print(f'User uploaded file "{filename}"')

!mv "{filename}" datasets/coco-stuff/

Saving labels.txt to labels (1).txt
User uploaded file "labels (1).txt"


In [22]:
!rm datasets/coco-stuff/labels.txt

In [38]:
!ls datasets/coco-stuff
!ls datasets/coco-stuff/images
!ls datasets/coco-stuff/annotations/


 annotations   images  'labels (1).txt'
train2017  val2017
train2017  val2017


In [32]:
open("datasets/coco-stuff/labels (1).txt").read().splitlines()

['0: unlabeled',
 '1: person',
 '2: bicycle',
 '3: car',
 '4: motorcycle',
 '5: airplane',
 '6: bus',
 '7: train',
 '8: truck',
 '9: boat',
 '10: traffic light',
 '11: fire hydrant',
 '12: street sign',
 '13: stop sign',
 '14: parking meter',
 '15: bench',
 '16: bird',
 '17: cat',
 '18: dog',
 '19: horse',
 '20: sheep',
 '21: cow',
 '22: elephant',
 '23: bear',
 '24: zebra',
 '25: giraffe',
 '26: hat',
 '27: backpack',
 '28: umbrella',
 '29: shoe',
 '30: eye glasses',
 '31: handbag',
 '32: tie',
 '33: suitcase',
 '34: frisbee',
 '35: skis',
 '36: snowboard',
 '37: sports ball',
 '38: kite',
 '39: baseball bat',
 '40: baseball glove',
 '41: skateboard',
 '42: surfboard',
 '43: tennis racket',
 '44: bottle',
 '45: plate',
 '46: wine glass',
 '47: cup',
 '48: fork',
 '49: knife',
 '50: spoon',
 '51: bowl',
 '52: banana',
 '53: apple',
 '54: sandwich',
 '55: orange',
 '56: broccoli',
 '57: carrot',
 '58: hot dog',
 '59: pizza',
 '60: donut',
 '61: cake',
 '62: chair',
 '63: couch',
 '64: p

In [ ]:
! ls datasets
#checking the stucture

train2017  val2017


In [42]:
# prompt: check the size and lenght of dataset
import os
train_images_path = 'datasets/coco-stuff/images/train2017'
val_images_path = 'datasets/coco-stuff/images/val2017'
train_masks_path = 'datasets/coco-stuff/annotations/train2017'
val_masks_path = 'datasets/coco-stuff/annotations/val2017'

train_image_files = os.listdir(train_images_path)
val_image_files = os.listdir(val_images_path)
train_mask_files = os.listdir(train_masks_path)
val_mask_files = os.listdir(val_masks_path)

print(f"Number of training images: {len(train_image_files)}")
print(f"Number of validation images: {len(val_image_files)}")
print(f"Number of training masks: {len(train_mask_files)}")
print(f"Number of validation masks: {len(val_mask_files)}")

# Optionally, you can also check the size of a single image and mask
if len(train_image_files) > 0:
  first_train_image_path = os.path.join(train_images_path, train_image_files[0])
  img = Image.open(first_train_image_path)
  print(f"Size of a training image: {img.size}")

if len(train_mask_files) > 0:
  first_train_mask_path = os.path.join(train_masks_path, train_mask_files[0])
  mask = Image.open(first_train_mask_path)
  print(f"Size of a training mask: {mask.size}")

Number of training images: 118287
Number of validation images: 5000
Number of training masks: 118287
Number of validation masks: 5000
Size of a training image: (640, 480)
Size of a training mask: (640, 348)


In [ ]:
# checking Size of dataset directory

!du -h datasets

846M	datasets/train2017
36M	datasets/val2017
882M	datasets
